<a href="https://colab.research.google.com/github/Collin-GP/colab-playground/blob/main/quickstarts/Search_Grounding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Gemini 2.0 - Grounding using Search as a tool

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Search_Grounding.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook you will learn how to use the new Google Search tool available in [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2), using both the unary API and the Multimodal Live API. Check out the docs to learn more about using [Search as a tool](https://ai.google.dev/gemini-api/docs/models/gemini-v2#search-tool).

Note that the previous version of this guide using Gemini models priori to 2.0 and the legacy SDK can still be found [here](https://github.com/google-gemini/cookbook/blob/gemini-1.5-archive/quickstarts/Search_Grounding.ipynb).

## Set up the SDK

This guide uses the [`google-genai`](https://pypi.org/project/google-genai) Python SDK to connect to the Gemini 2.0 models.

### Install SDK

The new **[Google Gen AI SDK](https://github.com/googleapis/python-genai)** provides programmatic access to Gemini 2 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs/models/gemini-v2) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://googleapis.github.io/python-genai/) or in the [Getting started](./Get_started.ipynb) notebook.

In [1]:
%pip install -q -U "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 6.5 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

### Select model and initialize SDK client

The client will pick up your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha` and use the Gemini 2.0 model.

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [4]:
from google import genai

client = genai.Client() # the API is automatically loaded from the environement variable

MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite-preview-06-17","gemini-2.0-flash","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Use Google Search
Search grounding is particularly useful for queries that require current information or external knowledge. Using Google Search, Gemini can access nearly real-time information and better responses.

In [8]:
from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents='I am running for the chicopee ma school board in 2025, what community events can i attend to maximize my contact with my voters?',
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"Response:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Response:
 To maximize your contact with voters for your 2025 Chicopee, MA school board campaign, consider attending a mix of official government meetings, recurring community gatherings, and annual events.

**Official Meetings & Public Forums:**

*   **Chicopee School Committee Meetings:** These are highly relevant as you're running for the school board. Meetings are typically held on a monthly basis during the school year, often on the second Wednesday evening of each month, though dates can be subject to change. They are usually held at the Chicopee City Hall Auditorium. You can find meeting notices and agendas on the city's calendar or the Chicopee Public Schools website. Some meetings may also offer a virtual (Zoom) participation option.
*   **Chicopee City Council Meetings:** These meetings are held on the first and third Tuesday of every month, except for July and August when they meet only on the first Tuesday. They occur at the City Council Chambers, 4th floor, City Hall Annex, immediately following the Mayor's Briefing which begins at 6:30 PM. Attending these meetings will allow you to understand broader city issues and connect with residents interested in local government.
*   **Other City Commission and Committee Meetings:** The city calendar lists various other meetings such as the Ambulance Commission, Housing Authority Board, Board of Assessors, and Water Resources Committee. Attending these can show your engagement with different aspects of city governance and community well-being.
*   **Chicopee Public Library Events:** The library often hosts various community activities, including "Genealogy Open Lab" and "Tech Goes Home" courses. Their social media also serves as a limited public forum.

**Recurring Community Events:**

*   **Chicopee Center Fresh Farmers Market:** This market runs weekly on Thursdays from 1 PM to 6 PM, from June through October, at Lucy Wisniowski Park (32 Chestnut St, Chicopee). It features vendors selling vegetables, agricultural products, prepared foods, and crafts, and is a great place to meet a diverse group of residents.
*   **Summer Band Concerts:** The Chicopee Parks & Recreation Department often hosts summer band concerts at the Aldenville Commons. For example, in 2025, there are concerts scheduled on various Tuesdays and Wednesdays in July and August.

**Annual and Seasonal Events (based on past occurrences and 2025 projections):**

*   **Memorial Day Parade:** Chicopee holds an annual Memorial Day Parade to honor fallen veterans.
*   **Chicopee Patriotic Parade and Fireworks:** The city has hosted an annual Patriotic Parade followed by a fireworks display, typically in late June. The fireworks for 2025 are scheduled for Sunday, June 29, at 9:30 PM at Szot Park.
*   **Chicopee Pride Festival:** The fifth annual Chicopee Pride Festival is scheduled for Saturday, June 14, 2025, from noon to 5 PM at Szot Park. This free event celebrates the LGBTQ+ community and features entertainment. A Pride Month flag raising event is also held at City Hall Flag Plaza on Friday, June 13, 2025.
*   **Strawberry Festivals:** The Second Annual Strawberry Festival is planned for Saturday, June 14, 2025, from 12 PM to 5 PM at McKinstry Market Garden. It features craft vendors, live music, farm animals, and food.
*   **Chicopee Irish Night:** This event, hosted by the Holyoke St. Patrick's Parade Committee, usually takes place in early March. In 2025, it's scheduled for Friday, March 7, at the Portuguese American Club. The committee also announces award recipients for the upcoming Holyoke St. Patrick's Parade in December, with a President's Dinner in February. The Holyoke St. Patrick's Parade itself, which Chicopee participates in, is scheduled for March 23, 2025.
*   **Faded Barber Lounge Back to School Event:** This community event, focused on schools, is listed on the Chicopee Chamber of Commerce calendar for Sunday, August 24, 2025.
*   **Hispanic Heritage Dominos Tournament:** Another event listed by the Chicopee Chamber of Commerce, scheduled for Saturday, August 23, 2025.
*   **Chicopee Chamber of Commerce Events:** Beyond the specific events listed, the Chamber of Commerce calendar is a good resource for community and networking events, such as their 2025 Membership Appreciation event on Wednesday, July 16, 2025.
*   **Walking Tours:** Various "Chicopee Walking Tours" covering themes like "Arts & Industry" and "A Trip to Old Cabotville" are sometimes offered, providing an opportunity to engage with residents interested in local history and culture.

**General Strategies:**

*   **Community Calendars:** Regularly check the "City Events Calendar" on the Chicopee, MA official website, the Chicopee Public Library's event calendar, the Chicopee Chamber of Commerce events calendar, and platforms like Eventbrite and AllEvents for upcoming activities.
*   **Police Department Community Outreach:** The Chicopee Police Department also lists community outreach events they offer, such as "Coffee with a Cop" and "Trunk or Treat" (seasonal), which could provide informal opportunities for interaction.
*   **Local Organizations and Cultural Groups:** Look for events hosted by specific community groups, cultural councils, or local associations that align with your interests and where you might find engaged voters. The Chicopee Cultural Council supports various arts and humanities projects in the community and holds monthly meetings.

By actively participating in these diverse events, you can connect with a broad range of Chicopee residents and gain a deeper understanding of their concerns and priorities as you campaign for the school board.

Search Query: ['recurring community events Chicopee MA', 'annual festivals Chicopee MA', 'Chicopee MA parades', 'Chicopee MA farmers markets', 'Chicopee MA cultural events', 'Chicopee MA city council meetings', 'Chicopee MA school committee meetings', 'Chicopee MA public forums', 'Chicopee MA community calendar', 'Chicopee MA local events 2025']
Search Pages: chicopeema.gov, chicopeeps.org, mass.edu, chicopeema.gov, chicopeetv.org, chicopeeps.org, chicopeema.gov, youtube.com, youtube.com, chicopeema.gov, patch.com, chicopeepubliclibrary.org, chicopeepubliclibrary.org, marketspread.com, buylocalfood.org, valleyopp.com, theq997.com, chicopeema.gov, chicopeema.gov, chicopeema.gov, chicopeema.gov, thereminder.com, macaronikid.com, allevents.in, holyokestpatricksparade.com, chicopeespc.com, thereminder.com, chicopeechamber.org, allevents.in, allevents.in, eventbrite.com, eventbrite.com, chicopeema.gov, allevents.in, eventbrite.com.au, chicopeema.gov, massculturalcouncil.org, meetup.com


The information provided may be outdated, as it is based on a knowledge cutoff. For the most current and authoritative details, please refer to the official [Model documentation](https://ai.google.dev/gemini-api/docs/models#gemini-2.5-pro), which includes the latest knowledge cutoff date.

You can see that running the same prompt without search grounding gives you outdated information:

In [10]:
from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents='I am running for the chicopee ma school board in 2025, when does mail in ballot open and close? consider primary and general elections if applicable',
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"Response:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Response:
 Massachusetts offers "no-excuse" early voting by mail for most local, state, and federal elections. While the exact dates for Chicopee's 2025 school board election primaries (if any) and the general election aren't explicitly listed, the information available provides a general framework for mail-in voting in Massachusetts.

Here's what you need to know for the Chicopee, MA School Board election in 2025:

**Mail-in Ballot Application Deadlines:**
*   Your application for a mail-in ballot must reach your local election officials by 5 p.m. on the 5th business day before the election.
*   For the General Municipal Election on Tuesday, November 4, 2025, the voter registration deadline is Saturday, October 25, 2025.

**Mail-in Ballot Return Deadlines:**
*   For most elections in Massachusetts, including municipal elections, your completed mail-in ballot must be received by your local election office or a drop box by 8:00 PM on Election Day to be counted.
*   The only exception is for the biennial state election in November, where ballots postmarked by Election Day have an additional three days to arrive. However, the school board election would fall under municipal elections, so the Election Day receipt deadline generally applies.

**Important Considerations:**
*   **Chicopee Specifics:** While state laws govern much of the mail-in ballot process, the specific dates for Chicopee's school board election (including any potential primary) will be set locally. It is crucial to confirm these dates with the Chicopee Registrar of Voters. You can find their contact information and a calendar of important dates on the City of Chicopee website.
*   **Receiving an Application:** The state will send a pre-addressed, postage-paid Vote by Mail application to every registered voter before each statewide election, typically 45 days beforehand. For local elections, like a school board election, you may need to proactively request an application.
*   **Applying for Your Ballot:** You can request your mail-in ballot online, by mail, or by email or fax. You can apply for mail-in ballots for all elections in a calendar year with a single application.
*   **Tracking Your Ballot:** You can track the status of your mail-in ballot through the state's website. This tool shows when your application was received, when your ballot was mailed, and when your completed ballot was received.
*   **Voting in Person:** If you apply to vote by mail but your ballot is not received, you can still vote in person on Election Day or at an early voting location. Measures are in place to prevent a ballot from being counted twice.

Search Query: ['chicopee ma school board election 2025 mail-in ballot dates', 'Massachusetts mail-in ballot deadlines 2025', 'Chicopee MA election calendar 2025']
Search Pages: watertown-ma.gov, sec.state.ma.us, vote411.org, brooklinema.gov, boston.gov, sec.state.ma.us, chicopeema.gov, chicopeema.gov, vote411.org
